In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-dm8b0zzs because the default path (/home/drifter/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [36]:
food_data = pd.read_csv('../raw_data/food_types_v2.csv')
country_data = pd.read_csv('../raw_data/clean_data.csv')
merged = pd.read_csv('../raw_data/merged_data.csv')
cpi = pd.read_csv('../raw_data/cpi_short.csv')

In [4]:
food_data.head()

,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal
0,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0
1,Oil (cooking),1.26,3.24,30.9,not found,116,0.0,3.0,144
2,Pulses,17.9,7.14,60.7,not found,393,0.0,107,393
3,Rice (high quality),7.19,3.85,75.5,not found,1.0,not found,10.0,365
4,Rice (low quality),7.19,3.85,75.5,not found,1.0,not found,10.0,365


In [5]:
merged.head()

,adm0_name,pt_name,um_name,mp_price,conv_rate,usd_rate,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal
0,Afghanistan,Retail,KG,50.344430,96.252560,0.523045,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0
1,Argentina,Retail,KG,114.238000,101.364698,1.127000,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0
2,Congo,Retail,KG,1078.371429,581.547941,1.854312,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0
3,Gambia,Retail,KG,27.529414,52.500260,0.524367,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0
4,Kyrgyzstan,Retail,KG,52.387667,84.797101,0.617800,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0


In [37]:
#mergin the merged data with the cpi of 2019

merged = pd.merge(left=merged,right=cpi,left_on='adm0_name', right_on='Country Name')
merged.head(1)

,adm0_name,pt_name,um_name,mp_price,conv_rate,usd_rate,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal,Unnamed: 0,Country Name,2019
0,Afghanistan,Retail,KG,50.34443,96.25256,0.523045,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0,2,Afghanistan,149.895975


In [41]:
merged.drop(columns=['Country Name', 'Unnamed: 0'], inplace=True)

In [42]:
merged.head(1)

,adm0_name,pt_name,um_name,mp_price,conv_rate,usd_rate,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal,2019
0,Afghanistan,Retail,KG,50.34443,96.25256,0.523045,Bread,9.43,3.59,49.2,5.34,477,not found,211,1130.0,149.895975


In [43]:
len(merged[merged['adm0_name'] == 'Gambia'])

54

In [44]:
merged['adm0_name'].unique()

array(['Afghanistan', 'Argentina', 'Congo', 'Gambia', 'Kyrgyzstan',
       'Peru', 'State of Palestine', 'Ecuador', 'Pakistan', 'Myanmar',
       'Colombia', 'Guinea-Bissau', 'Indonesia', 'Armenia',
       'Bassas da India', 'Bolivia', 'Democratic Republic of the Congo',
       'Djibouti', 'Egypt', 'Ethiopia', 'Guinea',
       'Iran  (Islamic Republic of)', 'Iraq', 'Japan', 'Jordan', 'Libya',
       'Mali', 'Mauritania', 'Rwanda', 'Sierra Leone', 'Sri Lanka',
       'Syrian Arab Republic', 'Tajikistan', 'Turkey', 'Yemen', 'Benin',
       'Chad', 'Angola', 'Bangladesh', 'Burundi',
       'Central African Republic', "Cote d'Ivoire",
       "Lao People's Democratic Republic", 'Nigeria', 'Nepal',
       'Kazakhstan', 'Mongolia', 'South Sudan', 'Belarus', 'Bhutan',
       'Cape Verde', 'Lebanon', 'Namibia', 'Swaziland', 'Malawi',
       'Uganda', 'Ghana', 'Thailand', 'Philippines', 'Cambodia', 'Kenya',
       'Somalia', 'Mozambique', 'Paraguay', 'Togo', 'Niger',
       'Burkina Faso', 'Mexi

In [45]:
#cecking for which country we have more types of food, so we will select that country to perform the model
length = []
for country in merged['adm0_name'].unique():
    length.append({'country': country, 'foods': len(merged[merged['adm0_name'] == country])})

In [46]:
pd.DataFrame(length).sort_values(by='foods', ascending=False)

,country,foods
10,Colombia,68
30,Sri Lanka,61
7,Ecuador,56
3,Gambia,54
35,Benin,50
...,...,...
48,Belarus,3
37,Angola,2
70,Liberia,2
57,Thailand,1


In [47]:
#we can see Colombia is the country with more data with 68 types of food

colombia_df = merged[merged['adm0_name'] == 'Colombia']

In [48]:
colombia_df.shape

(68, 16)

In [49]:
colombia_df.head()

,adm0_name,pt_name,um_name,mp_price,conv_rate,usd_rate,food_type,protein,fat,carb,sugar,sodium,cholesterol,calcium,kcal,2019
235,Colombia,Wholesale,KG,3147.391304,3910.98,0.804758,Rice (high quality),7.19,3.85,75.5,not found,1.0,not found,10.0,365,140.950374
236,Colombia,Wholesale,KG,2753.642857,3910.98,0.704080,Sugar,0.0,0.32,99.6,99.8,1.0,not found,1.0,385,140.950374
237,Colombia,Wholesale,KG,7377.058824,3910.98,1.886243,Wheat,15.1,2.73,71.2,not found,3.0,not found,38.0,1550.0,140.950374
238,Colombia,Wholesale,KG,17130.578703,3910.98,4.380124,Meat (beef),11.7,28.0,2.89,1.26,872,not found,15.0,1310.0,140.950374
239,Colombia,Wholesale,KG,936.285714,3910.98,0.239399,Potatoes,2.7,0.0,16.9,not found,0.0,0.0,0.0,74.0,140.950374


In [50]:
colombia_df['pt_name'].value_counts()

Wholesale    68
Name: pt_name, dtype: int64

In [51]:
#there are not retail prices in the colombian dataset

Declaring the y and X features

In [241]:
y=colombia_df['usd_rate']
X = colombia_df[['food_type','protein','fat','carb','sodium','calcium','kcal']]

In [242]:
X = X.replace('not found', np.nan)

### Preparing the imputing and the pipeline

In [243]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()

k_imputer = KNNImputer(n_neighbors=4)

In [244]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


num_trans = Pipeline([
    ('imputer', KNNImputer(n_neighbors=4)),
    ('sclaer', MinMaxScaler())
])

cat_trans = OneHotEncoder(handle_unknown='ignore', sparse=False)

preprocessor = ColumnTransformer([
    ('num_trans', num_trans,['protein','fat','carb','sodium','calcium','kcal']),
    ('cat_trans', cat_trans, ['food_type'])
])

#['protein','fat','carb','sodium','calcium','kcal','2019']

In [245]:
X_t =  preprocessor.fit_transform(X)
#X2_t =  preprocessor2.fit_transform(X2)
X_t

array([[0.08998748, 0.0385    , 0.75803213, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.0032    , 1.        , ..., 0.        , 0.        ,
        0.        ],
       [0.18898623, 0.0273    , 0.71485944, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.0065    , 0.06044177, ..., 0.        , 0.        ,
        0.        ],
       [0.00871089, 0.00425   , 0.03855422, ..., 0.        , 0.        ,
        0.        ],
       [0.18898623, 0.0273    , 0.71485944, ..., 0.        , 1.        ,
        0.        ]])

### Splitting the x and y

In [312]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [313]:
X_train, X_test, y_train, y_test = train_test_split(X_t, y, test_size=0.3)

### Basic linear regression

In [314]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

In [315]:
base_model = LinearRegression()

cv_results = cross_validate(base_model, X_train, y_train, cv=5, scoring = ['max_error','r2','neg_mean_squared_error','neg_mean_absolute_error'])
cv_results

{'fit_time': array([0.00215197, 0.00131917, 0.00389147, 0.00190496, 0.00160909]),
 'score_time': array([0.00219202, 0.00183821, 0.0044291 , 0.00271654, 0.00117397]),
 'test_max_error': array([-1.18088225, -4.87989614, -4.73361222, -3.08010775, -1.15271603]),
 'test_r2': array([-22.47787486,   0.07624611,   0.20566352,   0.04957641,
          0.30544593]),
 'test_neg_mean_squared_error': array([-0.73296811, -3.28298565, -2.7959454 , -1.32922636, -0.82071571]),
 'test_neg_mean_absolute_error': array([-0.83260868, -1.10138324, -1.00348108, -0.76174371, -0.88130004])}

In [316]:
base_results = {
    'test_max_error': cv_results['test_max_error'].mean(),
    'test_r2': cv_results['test_r2'].mean(),
    'test_neg_mean_squared_error': cv_results['test_neg_mean_squared_error'].mean(),
    'test_neg_mean_absolute_error': cv_results['test_neg_mean_absolute_error'].mean()
}
base_results

{'test_max_error': -3.0054428792890144,
 'test_r2': -4.368188577435199,
 'test_neg_mean_squared_error': -1.7923682458519845,
 'test_neg_mean_absolute_error': -0.9161033486079415}

In [317]:
base_model.fit(X_train, y_train)
base_model.score(X_test, y_test)

-1.8743363376510707

In [318]:
scores = cross_val_score(base_model, X_train, y_train, cv=10)
scores.mean()

-14.2274063776306

### Basic KNN regressor

In [319]:
from sklearn.neighbors import KNeighborsRegressor

k_model = KNeighborsRegressor(n_neighbors=5)

In [320]:
cv_results_k = cross_validate(k_model, X_train, y_train, cv=5, scoring = ['max_error','r2','neg_mean_squared_error','neg_mean_absolute_error'])

In [321]:
k_model_results = {
    'test_max_error': cv_results_k['test_max_error'].mean(),
    'test_r2': cv_results_k['test_r2'].mean(),
    'test_neg_mean_squared_error': cv_results_k['test_neg_mean_squared_error'].mean(),
    'test_neg_mean_absolute_error': cv_results_k['test_neg_mean_absolute_error'].mean()
}
k_model_results

{'test_max_error': -3.012300342190014,
 'test_r2': -0.6044676207026486,
 'test_neg_mean_squared_error': -1.5900675617047444,
 'test_neg_mean_absolute_error': -0.7397679039879977}

In [322]:
k_model.fit(X_train, y_train)
k_model.score(X_test, y_test)

-0.768468104982633

In [323]:
scores = cross_val_score(k_model, X_train, y_train, cv=10)
scores.mean()

-4.257965939090039

### Basic Ridge Regressor

In [324]:
from sklearn.linear_model import Ridge

r_model = Ridge(alpha=4)

In [325]:
cv_results_r = cross_validate(r_model, X_train, y_train, cv=5, scoring = ['max_error','r2','neg_mean_squared_error','neg_mean_absolute_error'])

In [326]:
r_model_results = {
    'test_max_error': cv_results_r['test_max_error'].mean(),
    'test_r2': cv_results_r['test_r2'].mean(),
    'test_neg_mean_squared_error': cv_results_r['test_neg_mean_squared_error'].mean(),
    'test_neg_mean_absolute_error': cv_results_r['test_neg_mean_absolute_error'].mean()
}
r_model_results

{'test_max_error': -3.0734461985644175,
 'test_r2': -5.732558470843849,
 'test_neg_mean_squared_error': -2.034454420115477,
 'test_neg_mean_absolute_error': -0.9888526073247308}

In [327]:
r_model.fit(X_train, y_train)
r_model.score(X_test, y_test)

-0.5907815411886241

In [328]:
scores = cross_val_score(r_model, X_train, y_train, cv=10)
scores.mean()

-17.587450704110868

### Basic SVR 

In [294]:
from sklearn.svm import SVR

svr_model = SVR(kernel='poly', C=0.2, epsilon=0.2)

In [295]:
cv_results_svr = cross_validate(svr_model, X_train, y_train, cv=5, scoring = ['max_error','r2','neg_mean_squared_error','neg_mean_absolute_error'])

In [296]:
svr_model_results = {
    'test_max_error': cv_results_svr['test_max_error'].mean(),
    'test_r2': cv_results_svr['test_r2'].mean(),
    'test_neg_mean_squared_error': cv_results_svr['test_neg_mean_squared_error'].mean(),
    'test_neg_mean_absolute_error': cv_results_svr['test_neg_mean_absolute_error'].mean()
}
svr_model_results

{'test_max_error': -2.7791871528915904,
 'test_r2': -0.19684598260481248,
 'test_neg_mean_squared_error': -1.698884270940071,
 'test_neg_mean_absolute_error': -0.8113052309125038}

In [297]:
svr_model.fit(X_train, y_train)
svr_model.score(X_test, y_test)

-0.04392194091464385

In [311]:
scores = cross_val_score(svr_model, X_train, y_train, cv=10)
scores.mean()

-0.23841508512903306

### Partial conclusions

- Selecting one country does not improve the metrics
- Within one country, selecting only nutrients does worsen the model
- Keep the food type as an X feature prevents the metrics to go negative

- The holdout method shows different results with great variation

- The best score has been obtained by the KNN regressor with a r2 of 0.42

# Performing a random search

In [329]:
X.shape

(68, 7)